# BPN

In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
df = pd.read_csv('preprocessed_1.csv')
df = df.dropna()
df = pd.get_dummies(df, columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'])
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df = df.drop(['Date'], axis=1)

In [3]:
# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_df = df[:train_size]
test_df = df[train_size:]

In [4]:
# Separate the target variable from the input variables
X_train = train_df.drop('RainTomorrow', axis=1)
y_train = train_df['RainTomorrow']
X_test = test_df.drop('RainTomorrow', axis=1)
y_test = test_df['RainTomorrow']

In [5]:
from keras.models import load_model
model_final = load_model('./models/backpropagation.h5')

In [6]:
y_pred = model_final.predict(X_test)
y_pred=(y_pred>0.5)

In [7]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [8]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
acc = accuracy_score(y_test, y_pred)
print("Accuracy:",round(acc,2))

Accuracy: 0.86


In [9]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[22738   566]
 [ 3645  2143]]
0.8552523030386361
              precision    recall  f1-score   support

           0       0.86      0.98      0.92     23304
           1       0.79      0.37      0.50      5788

    accuracy                           0.86     29092
   macro avg       0.83      0.67      0.71     29092
weighted avg       0.85      0.86      0.83     29092



In [39]:
from sklearn.metrics import cohen_kappa_score

y_prob = model_final.predict(X_test)  # get predicted probabilities
y_pred = (y_prob > 0.5)  # convert probabilities to binary labels
kappa = cohen_kappa_score(y_test, y_pred)
print(kappa)

0.48130079828785155


# RNN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,SimpleRNN,Input

In [2]:
import keras
from keras.models import Sequential

In [3]:
df = pd.read_csv("preprocessed_1.csv")
X = df.drop(["RainTomorrow", "Date"], axis=1)
Y = df["RainTomorrow"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size =0.2, stratify = Y, random_state = 0)

In [5]:
y_train

43447     0
140904    0
54615     0
109793    0
74462     0
         ..
106567    0
52791     1
56339     0
82299     1
115175    0
Name: RainTomorrow, Length: 116368, dtype: int64

In [6]:
sm=SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_res)))

The number of classes before fit Counter({0: 90866, 1: 25502})
The number of classes after fit Counter({0: 90866, 1: 90866})


In [7]:
import tensorflow as tf

X_train_res = tf.expand_dims(X_train_res, axis=1)

In [8]:
X_test = tf.expand_dims(X_test, axis=1)

In [9]:
# reshape your labels as 2d-tensor
y_train_res = np.asarray(y_train_res).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [10]:
num_features=X.shape[1]
num_features

23

In [11]:
from keras.models import load_model
model_final_rnn = load_model('./models/rnn.h5')

In [12]:
y_pred = model_final_rnn.predict(X_test)
y_pred=(y_pred>0.5)

In [13]:
y_pred

array([[False],
       [False],
       [ True],
       ...,
       [False],
       [ True],
       [False]])

In [14]:
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[19057  3660]
 [ 2055  4320]]
0.8035542417159357
              precision    recall  f1-score   support

         0.0       0.90      0.84      0.87     22717
         1.0       0.54      0.68      0.60      6375

    accuracy                           0.80     29092
   macro avg       0.72      0.76      0.74     29092
weighted avg       0.82      0.80      0.81     29092



In [16]:
from sklearn.metrics import cohen_kappa_score

y_prob = model_final_rnn.predict(X_test)  # get predicted probabilities
y_pred = (y_prob > 0.5)  # convert probabilities to binary labels
kappa = cohen_kappa_score(y_test, y_pred)
print(kappa)

0.47364285115939553


# MLP

In [123]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
import joblib
import pickle

In [124]:
df = pd.read_csv("preprocessed_1.csv")
pd.set_option("display.max_columns", None)
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Date_month,Date_day
0,2008-12-01,30,13.4,22.9,0.6,2.4,8.3,4.0,44.0,5.0,3.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,0.0,16.9,21.8,0,0,12,1
1,2008-12-02,30,7.4,25.1,0.0,3.6,10.0,2.0,44.0,0.0,6.0,4.0,22.0,44.0,25.0,1010.6,1007.8,7.0,1.0,17.2,24.3,0,0,12,2
2,2008-12-03,30,12.9,25.7,0.0,2.6,4.4,5.0,46.0,5.0,6.0,19.0,26.0,38.0,30.0,1007.6,1008.7,8.0,2.0,21.0,23.2,0,0,12,3
3,2008-12-04,30,9.2,28.0,0.0,14.6,8.9,11.0,24.0,13.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,0.0,5.0,18.1,26.5,0,0,12,4
4,2008-12-05,30,17.5,32.3,1.0,5.4,3.0,4.0,41.0,12.0,0.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0,12,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,49,2.8,23.4,0.0,1.4,7.8,15.0,31.0,13.0,13.0,13.0,11.0,51.0,24.0,1024.6,1020.3,1.0,1.0,10.1,22.4,0,0,6,21
145456,2017-06-22,49,3.6,25.3,0.0,7.6,13.5,0.0,22.0,13.0,2.0,13.0,9.0,56.0,21.0,1023.5,1019.1,8.0,5.0,10.9,24.5,0,0,6,22
145457,2017-06-23,49,5.4,26.9,0.0,6.8,11.0,3.0,37.0,13.0,3.0,9.0,9.0,53.0,24.0,1021.0,1016.8,8.0,1.0,12.5,26.1,0,0,6,23
145458,2017-06-24,49,7.8,27.0,0.0,2.6,13.2,12.0,28.0,11.0,2.0,13.0,7.0,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,0,0,6,24


In [125]:
X = df.drop(["RainTomorrow", "Date"], axis=1)
Y = df["RainTomorrow"]

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size =0.2, stratify = Y, random_state = 0)

In [127]:
y_train

43447     0
140904    0
54615     0
109793    0
74462     0
         ..
106567    0
52791     1
56339     0
82299     1
115175    0
Name: RainTomorrow, Length: 116368, dtype: int64

In [128]:
sm=SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_res)))

The number of classes before fit Counter({0: 90866, 1: 25502})
The number of classes after fit Counter({0: 90866, 1: 90866})


In [129]:
from sklearn.preprocessing import StandardScaler 
from sklearn.neural_network import MLPClassifier
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train_res)  
X_train_res = scaler.transform(X_train_res)  
# apply same transformation to test data
X_test = scaler.transform(X_test) 

In [130]:
with open('./models/mlp.pkl', 'rb') as file:
    mlp_model = pickle.load(file)

In [131]:
print("Accuracy:", mlp_model.score(X_test, y_test))

Accuracy: 0.8106008524680325


In [132]:
y_pred = mlp_model.predict(X_test)

In [133]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[19186  3531]
 [ 1979  4396]]
0.8106008524680325
              precision    recall  f1-score   support

           0       0.91      0.84      0.87     22717
           1       0.55      0.69      0.61      6375

    accuracy                           0.81     29092
   macro avg       0.73      0.77      0.74     29092
weighted avg       0.83      0.81      0.82     29092



In [134]:
from sklearn.metrics import cohen_kappa_score

y_prob = mlp_model.predict(X_test)  # get predicted probabilities
y_pred = (y_prob > 0.5)  # convert probabilities to binary labels
kappa = cohen_kappa_score(y_test, y_pred)
print(kappa)

0.4911282134116838


# RBF

In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import joblib
import pickle

In [18]:
df = pd.read_csv("preprocessed_1.csv")
pd.set_option("display.max_columns", None)
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Date_month,Date_day
0,2008-12-01,30,13.4,22.9,0.6,2.4,8.3,4.0,44.0,5.0,3.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,0.0,16.9,21.8,0,0,12,1
1,2008-12-02,30,7.4,25.1,0.0,3.6,10.0,2.0,44.0,0.0,6.0,4.0,22.0,44.0,25.0,1010.6,1007.8,7.0,1.0,17.2,24.3,0,0,12,2
2,2008-12-03,30,12.9,25.7,0.0,2.6,4.4,5.0,46.0,5.0,6.0,19.0,26.0,38.0,30.0,1007.6,1008.7,8.0,2.0,21.0,23.2,0,0,12,3
3,2008-12-04,30,9.2,28.0,0.0,14.6,8.9,11.0,24.0,13.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,0.0,5.0,18.1,26.5,0,0,12,4
4,2008-12-05,30,17.5,32.3,1.0,5.4,3.0,4.0,41.0,12.0,0.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0,12,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,49,2.8,23.4,0.0,1.4,7.8,15.0,31.0,13.0,13.0,13.0,11.0,51.0,24.0,1024.6,1020.3,1.0,1.0,10.1,22.4,0,0,6,21
145456,2017-06-22,49,3.6,25.3,0.0,7.6,13.5,0.0,22.0,13.0,2.0,13.0,9.0,56.0,21.0,1023.5,1019.1,8.0,5.0,10.9,24.5,0,0,6,22
145457,2017-06-23,49,5.4,26.9,0.0,6.8,11.0,3.0,37.0,13.0,3.0,9.0,9.0,53.0,24.0,1021.0,1016.8,8.0,1.0,12.5,26.1,0,0,6,23
145458,2017-06-24,49,7.8,27.0,0.0,2.6,13.2,12.0,28.0,11.0,2.0,13.0,7.0,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,0,0,6,24


In [19]:
X = df.drop(["RainTomorrow", "Date"], axis=1)
Y = df["RainTomorrow"]

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size =0.2, stratify = Y, random_state = 0)

In [21]:
y_train

43447     0
140904    0
54615     0
109793    0
74462     0
         ..
106567    0
52791     1
56339     0
82299     1
115175    0
Name: RainTomorrow, Length: 116368, dtype: int64

In [22]:
sm=SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_res)))

The number of classes before fit Counter({0: 90866, 1: 25502})
The number of classes after fit Counter({0: 90866, 1: 90866})


In [23]:
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train_res)  
X_train_res = scaler.transform(X_train_res)  
# apply same transformation to test data
X_test = scaler.transform(X_test) 

In [24]:
with open('./models/rbf.pkl', 'rb') as file:
    rbf_model = pickle.load(file)

In [25]:
y_pred = rbf_model.predict(X_test)

In [26]:
y_pred

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [27]:
print("Accuracy:", rbf_model.score(X_test, y_test))

Accuracy: 0.8106008524680325


In [28]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[19186  3531]
 [ 1979  4396]]
0.8106008524680325
              precision    recall  f1-score   support

           0       0.91      0.84      0.87     22717
           1       0.55      0.69      0.61      6375

    accuracy                           0.81     29092
   macro avg       0.73      0.77      0.74     29092
weighted avg       0.83      0.81      0.82     29092



In [29]:
from sklearn.metrics import cohen_kappa_score

y_prob = rbf_model.predict(X_test)  # get predicted probabilities
y_pred = (y_prob > 0.5)  # convert probabilities to binary labels
kappa = cohen_kappa_score(y_test, y_pred)
print(kappa)

0.4911282134116838
